<a href="https://colab.research.google.com/github/jaderAdriel/Clorofila-a/blob/main/Projeto_pibic_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread
!pip install matplotlib

**Import** the library, authenticate, and create the interface to Sheets

In [32]:
#@title Bilbiotecas
import pandas as pd
import matplotlib.pyplot as plt
from gspread import *
import numpy as np

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [26]:
#@title Worksheets { form-width: "20%" }
def get_worksheet(spreadsheet, word = ' '):

    listOfWorksheet = []
    word = word.upper()

    for worksheet in spreadsheet:
        if word in [i.upper() for i in worksheet.title.split()]:
            listOfWorksheet.append(worksheet)
            
    return listOfWorksheet


def get_years(worksheets):
    years = []
    for worksheet in worksheets:
        year = get_year_campaign(worksheet.title)
        year = year[0]
        if  year not in years:
            years.append(year)

    return sorted(years)


def get_year_campaign (title):

    title = title.split()
    year = None
    campaing = None

    for digit in title:
        if digit.isnumeric():
            if int(digit) > 2000 and int(digit) < 3000:
                year = digit
            elif int(digit) < 100 and int(digit) > 0:
                campaing = digit

    return year , campaing

In [27]:
#@title Validar dados { form-width: "20%", display-mode: "form" }
def valid_value(value):
    
    validValue = ''
    value = str(value)

    for i in value:
        if i == ',':
            validValue += '.'
        elif i == '<':
            validValue += ''
        else:
            validValue += i

    try:
        return float(validValue)
    except:
        return value

def valid_line(years_campaigns):

    empity_nows = []
    for i, yearCampaign in enumerate(years_campaigns):
        condicao = [yearCampaign['campanha 1'] == None,yearCampaign['campanha 2'] == None,
                    yearCampaign['campanha 3'] == None,yearCampaign['campanha 4'] == None]
        
        if condicao[0] and condicao[1] and condicao[2] and condicao[3]:
            empity_nows.append(i)

    empity_nows.sort(reverse=True)

    for i in empity_nows:
        years_campaigns.pop(i)

    return years_campaigns

In [30]:
#@title Default title text
def get_parameter(parameterName, worksheets, reservatory):

    years = get_years(worksheets)
    years_campaigns = get_years_campaigns(years)

    parameterUnitType, rowParameter, colUnity = [None,None,None]
    for worksheet in worksheets:
        try:
            line = worksheet.find(parameterName).row
            column = worksheet.find(reservatory).col
            information = valid_value(worksheet.cell(line, column).value)
        except:
            information = None

        year, campaign = get_year_campaign(worksheet.title)
        for i, yearCampaign in enumerate(years_campaigns):
            if yearCampaign['ano'] == str(year):
                if information == 'None' or information == 'sem dados':
                    information = None
                else:
                    if not rowParameter and information != None:
                            row = worksheet.find(parameterName).row 
                            col = worksheet.find('Unidade').col
                            parameterUnitType = worksheet.cell(row, col).value
                years_campaigns[i]['campanha '+campaign] = information


    return valid_line(years_campaigns), parameterUnitType

In [31]:
#@title Gŕaficos { form-width: "20%" }
def createGraph(dic, ylabel, title='parametro',  typeOfGraph='2'):
    if typeOfGraph == '1' or str(typeOfGraph).upper() == 'LINHA':
        lineGraph(dic, title, ylabel)
    else:
        subplotsGraph(dic, title, ylabel)


def lineGraph(dic, title, ylabel):
    campanhas = []
    anos = []
    for ano in dic:
        x = 1
        for campanha in ano.keys():
            if not campanha == 'ano':
                if ano[campanha]:
                    campanhas.append(ano[campanha])
                    anos.append(str(ano['ano'])+'.'+str(x))
              
                    x += 1
    x = anos
    y = campanhas
    plt.gcf().set_size_inches(20,8)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.plot(x, y, color='blue')
    plt.scatter(x, y, color='red')


def subplotsGraph(dic, title, ylabel):
    df = pd.DataFrame(dic)
    campanha = {
    'campanha 1' : list(df['campanha 1']),
    'campanha 2' : list(df['campanha 2']), 
    'campanha 3' : list(df['campanha 3']),
    'campanha 4' : list(df['campanha 4']),
    }

    x = np.arange(len(list(df['ano'])))

    fig, ax = plt.subplots()

    largura = 0.2

    campanha1 = ax.bar(x - largura*1.5 , campanha['campanha 1'],largura, label = 'campanha 1',color = 'turquoise')
    campanha2 = ax.bar(x - largura/2, campanha['campanha 2'],largura, label = 'campanha 2',color = 'tomato')
    campanha3 = ax.bar(x + largura/2, campanha['campanha 3'],largura, label = 'campanha 3',color = 'green')
    campanha4 = ax.bar(x + largura*1.5, campanha['campanha 4'],largura, label = 'campanha 4',color = 'blue')

    plt.figure(figsize=(15,10))
    ax.set_title(f'{title} ao longo dos anos')
    ax.set_ylabel(ylabel, fontsize = 12)
    ax.set_xlabel('Anos')
    ax.set_xticks(x)
    ax.set_xticklabels(list(df['ano']))

    plt.show()

In [47]:
#@title Spreadsheet { display-mode: "form" }
url1 = "https://docs.google.com/spreadsheets/d/1HxAY4p2IrmsxG5pAedtHASJg2fPxkOeeotw4wU6oqUw/edit#gid=956099665" #@param {type:"string"}
reservatory1 = "CND-CER-002" #@param {type:"string"}
url2 = "" #@param {type:"string"}
reservatory2 = "" #@param {type:"string"}
comparar = False #@param {type:"boolean"}

spreadsheet1 = gc.open_by_url(url1)
if comparar:
    spreadsheet2 = gc.open_by_url(url2)


In [51]:
parameter = "Temperatura" #@param ["Clorofila a", "Colif. termo", "Condutividade", "Salinidade", "Sólidos em suspensão", "Sólidos totais", "STD", "Temperatura", "Turbidez", "Alcalinidade total", "DBO", "DQO", "Nitrogênio Nitrato", "N-NH3", "N Total", "OD", "pH", "P Total", "% Saturação de Oxigênio Dissolvido", "Cloreto", "Ortofosfato solúvel", "Temperatura do Ar", "Transparência da Água"]

parameter_report = [None, None]
parameter_report[0] =  get_worksheet(spreadsheet1, 'Campanha')

dic1, parameterUnitType = get_parameter(parameter, parameter_report[0], reservatory1)

if comparar: 
    parameter_report[1] =  get_worksheet(spreadsheet2, 'Campanha');
    dic2, parameterUnitType = get_parameter(parameter, parameter_report[1], reservatory2)


In [ ]:
#@title Default title text
typeOfGraph = "Linha" #@param ["Barra", "Linha"]
if comparar:
    ...
else:
    title = f'{parameter} no {reservatory1}'
    createGraph(dic1, ylabel=parameterUnitType, title= title, typeOfGraph=typeOfGraph)